In [13]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
OutlineJSON_function = plugin["OutlineJSON"]

OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Xăng dầu năm 2023"))
print(OutlineJSON)


{'report': {'title': 'Báo cáo thị trường Xăng dầu năm 2023','introduction': 'Trong phần giới thiệu, báo cáo sẽ trình bày tình hình thị trường xăng dầu trong năm 2023 và những yếu tố ảnh hưởng đến thị trường.','body': {'headings': ['1. Tình hình thị trường xăng dầu hiện tại','2. Dự báo thị trường xăng dầu năm 2023','3. Ảnh hưởng của yếu tố chính trị và kinh tế đến thị trường','4. Các xu hướng mới trong ngành','5. Thách thức và cơ hội trong năm 2023']}, 'conclusion': 'Tóm lại, thị trường xăng dầu năm 2023 đang đối diện với nhiều thách thức nhưng cũng mang lại cơ hội cho các doanh nghiệp.','recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đưa ra các khuyến nghị cho các doanh nghiệp hoạt động trong ngành xăng dầu để thích nghi và phát triển trong năm 2023.'}}


In [14]:
from Plugins.utils import json_split
outline_dict = json_split(str(OutlineJSON))
outline_dict

{'title': 'Báo cáo thị trường Xăng dầu năm 2023',
 'introduction': 'Trong phần giới thiệu, báo cáo sẽ trình bày tình hình thị trường xăng dầu trong năm 2023 và những yếu tố ảnh hưởng đến thị trường.',
 'body': {'headings': ['1. Tình hình thị trường xăng dầu hiện tại',
   '2. Dự báo thị trường xăng dầu năm 2023',
   '3. Ảnh hưởng của yếu tố chính trị và kinh tế đến thị trường',
   '4. Các xu hướng mới trong ngành',
   '5. Thách thức và cơ hội trong năm 2023']},
 'conclusion': 'Tóm lại, thị trường xăng dầu năm 2023 đang đối diện với nhiều thách thức nhưng cũng mang lại cơ hội cho các doanh nghiệp.',
 'recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đưa ra các khuyến nghị cho các doanh nghiệp hoạt động trong ngành xăng dầu để thích nghi và phát triển trong năm 2023.'}

In [15]:
Introduction_function = plugin["Introduction"]
Introduction = await kernel.invoke(Introduction_function, sk.KernelArguments(introduction=outline_dict["introduction"]
                                                                            , reportLayout = str(OutlineJSON)))
print(Introduction)

Báo cáo thị trường Xăng dầu năm 2023 sẽ cung cấp cái nhìn tổng quan về tình hình thị trường xăng dầu trong năm tới và những yếu tố quan trọng ảnh hưởng đến thị trường. Chúng tôi sẽ trình bày về tình hình hiện tại của thị trường xăng dầu, dự báo cho năm 2023, ảnh hưởng của yếu tố chính trị và kinh tế, các xu hướng mới trong ngành, cũng như những thách thức và cơ hội mà thị trường đang đối diện. Đây sẽ là cơ sở để đưa ra những khuyến nghị hữu ích cho các doanh nghiệp hoạt động trong lĩnh vực xăng dầu, giúp họ thích nghi và phát triển trong năm 2023.


In [16]:
from Plugins.google_search import GoogleSearchPlugin
from Plugins.utils import token_count
import asyncio
TOKEN_LIMIT = 8000

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]
get_content_function = google_plugin["GetContentFromURL"]

column_function = plugin["Column"]
column_summ_function = plugin["ColumnSumm"]
headings = outline_dict["body"]["headings"]

async def columnist(heading):
    search_result = []
    tokens = 0
    search_result_urls = await kernel.invoke(google_search_function, sk.KernelArguments(keyword=heading, num_results = 15))
    search_result_urls = search_result_urls.value
    while True:
        if not search_result_urls:
            break
        url = search_result_urls.pop(0)
        content = await kernel.invoke(get_content_function, sk.KernelArguments(url = url))
        content = content.value
        new_tokens = token_count(content)
        if tokens + new_tokens > TOKEN_LIMIT:
            print(tokens)
            if tokens >= TOKEN_LIMIT//2:
                break
            else:
                continue
        tokens += new_tokens
        search_result.append(content)
    search_result = " ".join(search_result)
    column = await kernel.invoke(column_function, sk.KernelArguments(input = search_result, body_content = heading))
    column_summ = await kernel.invoke(column_summ_function, sk.KernelArguments(input = search_result, body_content = heading))
    return str(column), str(column_summ)

coros = []
for heading in headings:
    coros.append(columnist(heading + "(" + outline_dict["title"]+")"))

values = await asyncio.gather(*coros)
print(values)




5394
7009
7690
7937
7477
[('Trong bối cảnh thị trường xăng dầu hiện nay đang chịu ảnh hưởng lớn từ biến động giá thế giới, việc tăng giá xăng dầu trong nước là điều không tránh khỏi. Với việc giá dầu thô trên thị trường thế giới tăng mạnh, cùng với việc OPEC+ chưa có quyết định gia tăng sản lượng khai thác, đã khiến giá xăng dầu thành phẩm tăng đột ngột trong thời gian gần đây.\n\nNhìn chung, việc tăng giá xăng dầu sẽ ảnh hưởng đến nhiều khía cạnh của đời sống và kinh tế của người dân. Giá xăng dầu cao sẽ làm tăng chi phí vận chuyển hàng hóa, dẫn đến tăng giá các mặt hàng tiêu dùng. Đồng thời, các doanh nghiệp cũng sẽ phải đối mặt với áp lực chi phí cao hơn, có thể dẫn đến giảm lợi nhuận hoặc tăng giá sản phẩm và dịch vụ.\n\nTrong bối cảnh này, việc quản lý giá xăng dầu trở nên cực kỳ quan trọng. Chính phủ cần có những biện pháp linh hoạt để điều chỉnh giá xăng dầu sao cho hợp lý, không gây áp lực quá lớn cho người tiêu dùng và doanh nghiệp, đồng thời vẫn đảm bảo sự ổn định của thị trư

In [17]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])
    print(token_count(value[1]), "tokens")
    print(value[1])

775 tokens
Trong bối cảnh thị trường xăng dầu hiện nay đang chịu ảnh hưởng lớn từ biến động giá thế giới, việc tăng giá xăng dầu trong nước là điều không tránh khỏi. Với việc giá dầu thô trên thị trường thế giới tăng mạnh, cùng với việc OPEC+ chưa có quyết định gia tăng sản lượng khai thác, đã khiến giá xăng dầu thành phẩm tăng đột ngột trong thời gian gần đây.

Nhìn chung, việc tăng giá xăng dầu sẽ ảnh hưởng đến nhiều khía cạnh của đời sống và kinh tế của người dân. Giá xăng dầu cao sẽ làm tăng chi phí vận chuyển hàng hóa, dẫn đến tăng giá các mặt hàng tiêu dùng. Đồng thời, các doanh nghiệp cũng sẽ phải đối mặt với áp lực chi phí cao hơn, có thể dẫn đến giảm lợi nhuận hoặc tăng giá sản phẩm và dịch vụ.

Trong bối cảnh này, việc quản lý giá xăng dầu trở nên cực kỳ quan trọng. Chính phủ cần có những biện pháp linh hoạt để điều chỉnh giá xăng dầu sao cho hợp lý, không gây áp lực quá lớn cho người tiêu dùng và doanh nghiệp, đồng thời vẫn đảm bảo sự ổn định của thị trường.

Ngoài ra, việc 

In [18]:
conclusion_function = plugin["Conclusion"]
summContent = " ".join([value[1] for value in values])
conclusion = await kernel.invoke(conclusion_function, sk.KernelArguments(conclusion = outline_dict["conclusion"], summContent = summContent))
print(str(conclusion))


Trong bối cảnh giá xăng dầu thế giới tăng mạnh do nhu cầu tăng cao và các yếu tố khác như việc OPEC+ chưa gia tăng sản lượng khai thác, thị trường xăng dầu trong nước cũng đang phải đối mặt với áp lực tăng giá. Việc điều chỉnh giá xăng dầu trong nước là một biện pháp cần thiết để bảo đảm ổn định thị trường và hỗ trợ người tiêu dùng và doanh nghiệp. Trong thời gian gần đây, Bộ Công Thương đã quyết định ngừng trích lập Quỹ Bình ổn giá xăng dầu đối với một số mặt hàng như xăng RON95, dầu diesel, dầu hỏa và tăng chi sử dụng Quỹ BOG đối với xăng E5RON92, dầu diesel và dầu hỏa. Điều này giúp giảm áp lực tăng giá xăng dầu trong nước so với giá thế giới. Việc điều chỉnh giá bán xăng dầu cũng được thực hiện để đảm bảo không tăng quá mức so với giá cơ sở hiện hành. Đồng thời, việc duy trì mức chênh lệch giá giữa xăng sinh học E5RON92 và xăng khoáng RON95 cũng nhằm khuyến khích sử dụng nhiên liệu sinh học để bảo vệ môi trường. Tuy nhiên, việc điều chỉnh giá xăng dầu cần được thực hiện cẩn thận và

In [19]:
print(token_count(str(conclusion)))

2261


In [20]:
recommendation_function = plugin["Recommendation"]
summContent = " ".join([value[1] for value in values])
recommendation = await kernel.invoke(recommendation_function, sk.KernelArguments(recommendation = outline_dict["recommendations"], summContent = summContent))
print(str(recommendation))
print(token_count(str(recommendation)))

Based on the analysis presented in the report, we recommend the following strategies for businesses operating in the oil and gas industry to adapt and thrive in 2023:

1. Stay Informed and Monitor Market Trends: It is crucial for businesses to stay informed about global oil and gas market trends, including factors such as geopolitical tensions, economic conditions, and supply-demand dynamics. By closely monitoring these trends, businesses can make informed decisions and adjust their strategies accordingly to mitigate risks and capitalize on opportunities.

2. Diversify Supply Chains: Given the uncertainties in the global market, businesses should consider diversifying their supply chains to reduce dependence on any single source. Exploring new suppliers and markets can help mitigate risks associated with disruptions in the supply chain and ensure a stable flow of resources.

3. Embrace Technological Advancements: The integration of technologies such as Artificial Intelligence (AI), Blo